# Nettoyage et Prétraitement des Données pour la Détection de Fausses Nouvelles

## Introduction

Ce notebook détaille les étapes cruciales de nettoyage et de prétraitement des données textuelles issues des ensembles de nouvelles "Fake.csv" et "True.csv". L'objectif principal est de transformer le texte brut en un format structuré et normalisé, propice à l'extraction de caractéristiques (features) pertinentes pour l'entraînement de modèles d'apprentissage automatique capables de détecter les fausses nouvelles.

Le processus comprend plusieurs phases :
1.  **Chargement et exploration initiale des données** : Pour comprendre la structure et le contenu des jeux de données.
2.  **Gestion des mots vides (stopwords)** : Création d'une liste exhaustive de mots vides à partir de multiples sources pour les éliminer du texte.
3.  **Traitement par lots (chunks)** : Application d'un pipeline de nettoyage (passage en minuscules, suppression de la ponctuation et des caractères non pertinents, lemmatisation) de manière efficace en termes de mémoire.
4.  **Nettoyage final des DataFrames** : Gestion des valeurs manquantes et suppression des colonnes inutiles.
5.  **Extraction et sauvegarde des caractéristiques TF-IDF** : Vectorisation des textes en utilisant TF-IDF et calcul de la similarité cosinus entre titres et textes.
6.  **Extraction et sauvegarde des caractéristiques Doc2Vec** : Apprentissage de plongements de documents (Doc2Vec) et calcul de la similarité cosinus.
7.  **Combinaison et sauvegarde des ensembles de caractéristiques finaux** : Préparation des matrices de caractéristiques pour la phase de modélisation.

Chaque étape vise à améliorer la qualité des données d'entrée pour les modèles, en réduisant le bruit et en standardisant le format du texte.

## Data Cleaning (racinisation, suppression des stop words et ponctuation)

## 1. Initialisation et Chargement des Données

### Importation des Bibliothèques
Cette section commence par l'importation des bibliothèques Python nécessaires pour la manipulation des données, le traitement du langage naturel (NLP) et l'apprentissage automatique.
gensim.models.doc2vec import Doc2Vec, TaggedDocument


### Configuration Initiale de spaCy et NLTK
Le modèle `en_core_web_sm` de spaCy est chargé pour les opérations NLP, et les mots vides (stopwords) anglais de NLTK sont téléchargés et affichés à titre d'exemple.


### Chargement des Jeux de Données Bruts
Les jeux de données `Fake.csv` et `True.csv` sont chargés dans des DataFrames Pandas. Une première inspection est effectuée à l'aide de la méthode `.head()` pour visualiser les premières lignes et comprendre la structure des données (colonnes: `title`, `text`, `subject`, `date`).
*Note : Dans le code initial, `Fake.csv` est chargé pour `document_True_df` également. Pour la suite du traitement, il est supposé que `True.csv` est bien utilisé pour les "vraies nouvelles".

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump
import string
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
import re 
import spacy
import joblib 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [2]:
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
document_Fake_df = pd.read_csv("Fake.csv")
document_True_df = pd.read_csv("Fake.csv") 
(document_Fake_df.head())

 

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
(document_True_df.head())

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


## Combinaison et préparation des stopwords

## 2. Combinaison et Préparation des Mots Vides (Stopwords)

Afin d'éliminer efficacement les mots courants qui n'apportent que peu de valeur sémantique pour la classification, une liste complète de mots vides est construite en agrégeant ceux de plusieurs bibliothèques populaires : NLTK, spaCy, scikit-learn et Gensim.

### Fonctions Utilitaires
- `clean_word(word)`: Une fonction pour nettoyer un mot en ne conservant que les caractères alphabétiques et en le convertissant en minuscules.
- `get_combined_stopwords()`: Combine les listes de mots vides des différentes sources, nettoie chaque mot vide à l'aide de `clean_word`, et retourne une liste triée unique de mots vides.

Une instance de spaCy est chargée spécifiquement pour cette étape, en désactivant les composants non nécessaires (`parser`, `ner`, `lemmatizer`) pour optimiser le chargement.
t(s_words)

La variable globale `s_words` contient désormais la liste exhaustive de mots vides qui sera utilisée dans les étapes de nettoyage ultérieures.

In [7]:
import re
from nltk.corpus import stopwords as nltk_stopwords
import spacy
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stopwords
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords

# 1. Initialisation rapide de spaCy (sans composants inutiles)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "lemmatizer"])

def clean_word(word):
    """Nettoie le mot : conserve uniquement les lettres alphabétiques et met en minuscule"""
    return re.sub(r'[^a-z]', '', word.lower())

def get_combined_stopwords():
    """Combine les stopwords de toutes les sources et les nettoie"""
    # Sources de stopwords
    sources = {
        "nltk": set(nltk_stopwords.words('english')),
        "spacy": set(nlp.Defaults.stop_words),
        "sklearn": set(sklearn_stopwords),
        "gensim": set(gensim_stopwords)
    }
    
    # Combinaison et nettoyage
    combined = set()
    for source_words in sources.values():
        for word in source_words:
            cleaned = clean_word(word)  # Nettoie le mot
            if cleaned and len(cleaned):  # Ignore les mots vides ou trop courts
                combined.add(cleaned)
    
    return sorted(combined)
s_words = get_combined_stopwords()
print((s_words))

## Traitement des dataframes par chunks

## 3. Traitement des DataFrames par Lots (Chunks)

Pour gérer efficacement la mémoire lors du traitement de potentiellement grands volumes de texte, un pipeline de nettoyage est appliqué aux DataFrames par lots (chunks).

### Initialisation de spaCy pour la Lemmatisation
Une instance du modèle `en_core_web_sm` de spaCy est chargée. Pour la lemmatisation (`token.lemma_`), il est important que le composant "lemmatizer" soit actif. L'initialisation `nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])` active le lemmatiseur par défaut tout en désactivant les autres composants non requis pour cette tâche, optimisant ainsi les performances.

### Fonction `process_dataframe_in_chunks`
Cette fonction prend un chemin de fichier CSV, le nom de la colonne textuelle à traiter, la taille des lots et un chemin de sortie. Pour chaque lot :
1.  Le texte est converti en minuscules.
2.  Les caractères non alphabétiques (sauf les espaces) sont supprimés via une expression régulière.
3.  Les documents du lot sont traités par `nlp.pipe()` de spaCy pour une tokenisation et une lemmatisation efficaces.
4.  Pour chaque document traité, les tokens sont filtrés : les mots vides (contenus dans `s_words`) et la ponctuation sont supprimés. Les tokens de longueur 1 sont également ignorés. Les lemmes des tokens restants sont joints pour former le texte nettoyé.
5.  Les résultats du lot sont écrits dans le fichier de sortie (en mode écriture pour le premier lot, puis en mode ajout pour les suivants).


In [ ]:
import pandas as pd
import re
import spacy
from tqdm import tqdm
import re
import spacy


# 1. Initialisation rapide de spaCy (sans composants inutiles)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "lemmatizer"])

# Chargement du modèle une seule fois
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Traitement par petits lots pour préserver la mémoire
def process_dataframe_in_chunks(file_path, text_column, chunk_size=1000, output_path="dataset_processed_all_fake.csv"):
    global s_words
    # Initialiser le fichier de sortie
    first_chunk = True
    
    # Traiter le fichier par morceaux
    for chunk in tqdm(pd.read_csv(file_path, chunksize=chunk_size)):
        # Nettoyage basique (vectorisé)
        chunk[text_column] = chunk[text_column].str.lower()
        chunk[text_column] = chunk[text_column].str.replace(r'[^a-z\s]', '', regex=True)
        
        # Traitement spaCy (sans parallélisation excessive)
        docs = list(nlp.pipe(chunk[text_column], batch_size=100))
        
        # Extraire les tokens filtrés
        chunk[f"processed_{text_column}"] = [
            " ".join(token.lemma_ for token in doc 
                    if not token.text in s_words and not token.is_punct and len(token.text) > 1)
            for doc in docs
        ]
        
        # Écrire les résultats sans tout garder en mémoire
        mode = 'w' if first_chunk else 'a'
        header = first_chunk
        chunk.to_csv(output_path, mode=mode, header=header, index=False)
        first_chunk = False
        
        # Libérer la mémoire explicitement
        del docs
        
# Exécution
process_dataframe_in_chunks("Fake.csv", text_column="text", chunk_size=500,output_path = "dataset_processed.csv")
process_dataframe_in_chunks("dataset_processed.csv", text_column="title", chunk_size=500,output_path = "dataset_processed_all_fake.csv")
df = pd.read_csv("dataset_processed_all_fake.csv")
del df["title"]
del df["text"]
df.to_csv("dataset_Fake_final.csv")


In [11]:
process_dataframe_in_chunks("True.csv", text_column="text", chunk_size=500,output_path = "dataset_processed_true.csv")
process_dataframe_in_chunks("dataset_processed_true.csv", text_column="title", chunk_size=500,output_path = "dataset_processed_all_true.csv")
df = pd.read_csv("dataset_processed_all_true.csv")
del df["title"]
del df["text"]
df.to_csv("dataste_True_final.csv")

# Tokenization TF-IDF + similarité cosinus

## 5. Vectorisation TF-IDF et Calcul de Similarité

### Initialisation et Entraînement du Vectoriseur TF-IDF
La technique TF-IDF (Term Frequency-Inverse Document Frequency) est utilisée pour convertir les textes prétraités en une représentation numérique vectorielle.
- Un `TfidfVectorizer` de la bibliothèque scikit-learn est initialisé. Les paramètres clés sont :
    - `ngram_range=(1,2)` : pour considérer à la fois les mots individuels (unigrammes) et les paires de mots consécutifs (bigrammes) comme des caractéristiques.
    - `min_df=50` : pour ignorer les termes qui apparaissent dans moins de 50 documents, aidant à filtrer le bruit et à réduire la dimensionnalité.
- Un DataFrame `df_entrainement` est créé en concaténant les colonnes `processed_text` et `processed_title` des deux jeux de données (`dataset_fake` et `dataset_true`). Cette collection de tous les textes disponibles sert à construire le vocabulaire du vectoriseur TF-IDF.
- Le vectoriseur est ensuite "entraîné" (ajusté) sur ces données combinées avec la méthode `.fit()`.
- Pour une réutilisation future sans ré-entraînement, le vectoriseur ajusté est sauvegardé sur disque à l'aide de `joblib.dump()` et immédiatement rechargé avec `joblib.load()`. La taille du vocabulaire appris (nombre de caractéristiques uniques) est affichée.

```python
tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=50)

# Création du DataFrame d'entraînement pour le vocabulaire TF-IDF

In [4]:
dataset_fake = pd.read_csv("dataset_Fake_final.csv")
dataset_true = pd.read_csv("dataset_True_final.csv")

In [5]:
dataset_fake.isna().sum()

Unnamed: 0           0
subject              0
date                 0
processed_text     627
processed_title      0
dtype: int64

In [ ]:
del dataset_fake["Unnamed: 0"]
del dataset_true["Unnamed: 0"]
del dataset_true["Unnamed: 0.1"]

In [6]:
dataset_fake.dropna(inplace=True)
dataset_true.dropna(inplace=True)

In [7]:
dataset_fake.shape

(22854, 5)

In [3]:
tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=50)

In [31]:
df_entrainement = pd.DataFrame()
df_entrainement = pd.concat([df_entrainement,dataset_fake["processed_text"],dataset_fake["processed_title"],
           dataset_true["processed_text"],dataset_true["processed_title"]],ignore_index = True)

In [32]:
tf_idf_vectorizer.fit(df_entrainement[0])

In [ ]:
joblib.dump(tf_idf_vectorizer,"tf_idf_vectorizer.pkl")

In [34]:
tf_idf_vectorizer = joblib.load("tf_idf_vectorizer.pkl")
len(tf_idf_vectorizer.vocabulary_)

In [35]:
title_fake_vect = tf_idf_vectorizer.transform(dataset_fake["processed_title"])
text_fake_vect = tf_idf_vectorizer.transform(dataset_fake["processed_text"])
title_true_vect = tf_idf_vectorizer.transform(dataset_true["processed_title"])
text_true_vect = tf_idf_vectorizer.transform(dataset_true["processed_text"])

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp

# Exemple sur les fake news :
similarity_fake = cosine_similarity(title_fake_vect, text_fake_vect).diagonal()
# Exemple sur les true news :
similarity_true = cosine_similarity(title_true_vect, text_true_vect).diagonal()
# Maintenant, tu veux ajouter cette similarité en tant que nouvelle "colonne" (feature)
# On transforme similarity en matrice sparse compatible
similarity_fake_vect = sp.csr_matrix(similarity_fake).T  # transpose pour en faire une colonne
similarity_true_vect = sp.csr_matrix(similarity_true).T
# Puis on concatène horizontalement
fake_features = sp.hstack([title_fake_vect, text_fake_vect, similarity_fake_vect])
true_features = sp.hstack([title_true_vect, text_true_vect, similarity_true_vect])

print(fake_features.shape)  # (nombre d'articles, taille_title + taille_text + 1)
print(true_features.shape)

In [80]:
from scipy.sparse import save_npz

save_npz('fake_features.npz', fake_features)
save_npz('true_features.npz', true_features)

In [4]:
from scipy.sparse import load_npz

fake_features = load_npz('fake_features.npz')
true_features = load_npz('true_features.npz')

print(fake_features.shape)  # (nombre d'articles, taille_title + taille_text + 1)
print(true_features.shape)


(22854, 43504)
(21416, 43504)


# Doc2Vec feature extraction + similarité cosinus (Doc2Vec)

## 6. Extraction de Caractéristiques Doc2Vec et Calcul de Similarité

En complément de TF-IDF, la méthode Doc2Vec est utilisée pour apprendre des représentations vectorielles (plongements) de documents qui capturent mieux la sémantique.

### Entraînement du Modèle Doc2Vec
- Les documents utilisés pour entraîner le vectoriseur TF-IDF (`df_entrainement_content` qui contient tous les titres et textes prétraités) sont d'abord formatés en objets `TaggedDocument`. Chaque document est associé à un tag unique (ici, un identifiant numérique sous forme de chaîne).
- Un modèle `Doc2Vec` de la bibliothèque Gensim est initialisé avec des paramètres tels que `vector_size=100` (dimension des vecteurs), `window=2` (taille de la fenêtre de contexte), `min_count=1` (fréquence minimale des mots), `workers=4` (threads pour l'entraînement) et `epochs=100` (nombre de passes sur les données).
- Le vocabulaire du modèle Doc2Vec est construit à partir des `tagged_data`.
- Le modèle est ensuite entraîné sur ces mêmes `tagged_data`.
- Une fois l'entraînement terminé, le modèle Doc2Vec est sauvegardé sur disque et immédiatement rechargé pour une utilisation ultérieure.

In [ ]:

# Tagging each document (tag can be any unique identifier, here we're using integers)
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(df_entrainement[0])]

In [64]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Initialize the model
model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=100)

# Build the vocabulary
model.build_vocab(tagged_data)

# Train the model
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


In [68]:
# Sauvegarder le modèle
model.save("mon_modele_doc2vec.model")

In [10]:
model = Doc2Vec.load("mon_modele_doc2vec.model")

## Création et sauvegarde des features finales

In [11]:
def text_to_decvect(text):
 vector = model.infer_vector(text.split())
# Afficher le vecteur      
 return vector

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc

def infer_vectors_chunked(dataset, column_name, chunk_size=1000):
    """
    Traite un DataFrame déjà en mémoire par morceaux pour éviter les problèmes de mémoire
    
    Args:
        dataset: DataFrame pandas contenant les données
        column_name: Nom de la colonne à traiter
        chunk_size: Taille des morceaux à traiter
    
    Returns:
        Array numpy avec les vecteurs inférés
    """
    total_rows = len(dataset)
    all_vectors = []
    
    # Calculer le nombre de morceaux
    num_chunks = (total_rows + chunk_size - 1) // chunk_size
    
    # Traiter par morceaux
    for i in tqdm(range(0, total_rows, chunk_size), desc=f"Traitement de {column_name}", total=num_chunks):
        # Extraire un morceau
        end_idx = min(i + chunk_size, total_rows)
        chunk = dataset.iloc[i:end_idx]
        
        # Appliquer la fonction d'inférence vectorielle
        chunk_vectors = np.array([text_to_decvect(doc) for doc in chunk[column_name]])
        
        # Stocker les résultats
        all_vectors.append(chunk_vectors)
        
        # Afficher la progression
        print(f"Morceau {i//chunk_size + 1}/{num_chunks} traité, forme: {chunk_vectors.shape}")
        
        # Libérer la mémoire
        del chunk_vectors
        gc.collect()
    
    # Concaténer tous les vecteurs
    if all_vectors:
        result = np.vstack(all_vectors)
        print(f"Traitement terminé pour {column_name}. Forme finale: {result.shape}")
        return result
    else:
        return np.array([])

# Utiliser la fonction sur vos données
title_fake_docvect = infer_vectors_chunked(dataset_fake, "processed_title", chunk_size=500)
text_fake_docvect = infer_vectors_chunked(dataset_fake, "processed_text", chunk_size=500)
title_true_docvect = infer_vectors_chunked(dataset_true, "processed_title", chunk_size=500)
text_true_docvect = infer_vectors_chunked(dataset_true, "processed_text", chunk_size=500)

### Inférence des Vecteurs Doc2Vec et Calcul de la Similarité Cosinus
- Une fonction `text_to_d2v_vector(text, model_instance)` est définie (ou réutilisée/adaptée de `text_to_decvect`) pour inférer le vecteur Doc2Vec d'un texte donné. Elle prend le texte et l'instance du modèle Doc2Vec entraîné en arguments et utilise `model_instance.infer_vector()`.
- La fonction `infer_vectors_chunked` (définie précédemment dans le notebook) est utilisée pour générer les vecteurs Doc2Vec pour les colonnes `processed_title` et `processed_text` des DataFrames `dataset_fake` et `dataset_true`. Cette fonction doit être adaptée pour passer l'instance `model_d2v` à la fonction d'inférence vectorielle interne.
- Après avoir obtenu les vecteurs Doc2Vec pour les titres et les textes, la similarité cosinus est calculée entre chaque paire titre-texte.
- Ces scores de similarité sont transformés en matrices creuses d'une seule colonne.

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp

# Exemple sur les fake news :
similarity_fake = cosine_similarity(title_fake_docvect, text_fake_docvect).diagonal()
# Exemple sur les true news :
similarity_true = cosine_similarity(title_true_docvect, text_true_docvect).diagonal()
# Maintenant, tu veux ajouter cette similarité en tant que nouvelle "colonne" (feature)
# On transforme similarity en matrice sparse compatible
similarity_fake_doc_vect = sp.csr_matrix(similarity_fake).T  # transpose pour en faire une colonne
similarity_true_doc_vect = sp.csr_matrix(similarity_true).T
# Puis on concatène horizontalement
fake_features = sp.hstack([fake_features,similarity_fake_doc_vect])
true_features = sp.hstack([true_features, similarity_true_doc_vect])

from scipy.sparse import save_npz

save_npz('fake_features.npz', fake_features)
save_npz('true_features.npz', true_features)
print(fake_features.shape)  # (nombre d'articles, taille_title + taille_text + 1)
print(true_features.shape)

In [19]:
fake_supp_features = sp.hstack([title_fake_docvect,text_fake_docvect,similarity_fake_doc_vect])
true_supp_features = sp.hstack([title_true_docvect,text_true_docvect,similarity_true_doc_vect])
save_npz('fake_supp_features.npz', fake_supp_features)
save_npz('true_supp_features.npz', true_supp_features)

Ce notebook a accompli un travail de préparation de données substantiel, transformant des textes bruts en divers ensembles de caractéristiques numériques. Ces caractéristiques, notamment `fake_features.npz` et `true_features.npz` (basées sur TF-IDF et augmentées par des similarités) ainsi que `fake_supp_features.npz` et `true_supp_features.npz` (basées sur Doc2Vec), sont maintenant prêtes pour la phase de modélisation en vue de la détection de fausses nouvelles.